In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import requests
import json
import math
from numpy import random
from time import sleep

In [ ]:
main_category_names = []
sub_category_urls = []
sub_category_names = []
final_category_urls = []
final_category_names = []
product_urls = []
discounts = []
brand_names = []
product_skus = []
currency_codes = []
seller_names = []
product_names = []
product_prices = []
chat_responses = []
positive_sellers = []
ship_on_times = []
stars = []
reviews = []
delivery_times = []
delivery_fees = []

## categories

In [ ]:
def get_sub_category_url():
    
    source = requests.get("https://www.daraz.pk/")
    sleeptime = random.uniform(3, 8)
    sleep(sleeptime)
    
    soup = BeautifulSoup(source.content, "html.parser")
    urls = soup.find_all("ul", {"class": "lzd-site-menu-sub"})

    for url in urls:

        c = url.find_all("li", {"class": "lzd-site-menu-sub-item"})
        d = url.find_all("li", {"class": "sub-item-remove-arrow"})

        for i in c:
            sub_cat_url = i.find('a').get('href')
            sub_category_urls.append(sub_cat_url)
        for i in d:
            sub_cat_url = i.find('a').get('href')
            sub_category_urls.append(sub_cat_url)
    
    print(len(sub_category_urls))


In [ ]:
def get_final_category_url():
    link = sub_category_urls[0]
    url =f"https:{link}"

    source = requests.get(url)
    sleeptime = random.uniform(3, 8)
    sleep(sleeptime)
    soup = BeautifulSoup(source.content, "html.parser")
    a = soup.find_all("script")
    a = a[7:131]
    for i in a:
        try:
            i = str(i)
            start = i.find(">")
            dic = i[start+1:-9]
            json_data = json.loads(dic)
            for each in json_data:
                try:
                    url = each["childCategoryUrl"]
                    if url == '':
                        pass
                    else:
                        final_category_urls.append(url)
                except Exception as e:
                    print(f"Got exception in final_category_url: {str(e)}")
        except:
            pass
    print(len(final_category_urls))


## Products

In [ ]:
def get_product_url():

    for link in final_category_urls:
        try:
            url = f"https:{link}"
            print(url)
            source = requests.get(url, 'html.parser')
            sleeptime = random.uniform(3, 8)
            sleep(sleeptime)
    
            soup = BeautifulSoup(source.content, "html.parser")
            script = soup.find_all("script")
            t =str(script[3])
            start = t.find("totalResults")
            end = t.find("pageSize")
            item =int(t[start:end][15:-3])
            total_page = math.ceil(item/40)
            page = 1

            for page in range(page, total_page+1):
                page_url = f"https:{link}?page={page}"
                print(page_url)
                try:

                    source = requests.get(page_url)
                    sleeptime = random.uniform(1, 5)
                    sleep(sleeptime)

                    soup = BeautifulSoup(source.content, "html.parser")
                    try:
                        script = str(soup.find_all("script")[-1])
                        start = script.find(">")
                        dic_of_data = script[start+1:-9]
                        json_data = json.loads(dic_of_data)

                        for each in json_data["itemListElement"]:
                            product_url = each["url"]
                            print(f"Product Url: {product_url}")
                            product_urls.append(product_url)

                    except Exception as e:
                        print(f"Got exception in getting product_url: {str(e)}")
                except Exception as e:
                    print(f"Got exception in page_url: {str(e)}")
        except Exception as e:
            print(f"Got exception in grand_cat_link: {str(e)}")
    print(len(product_urls))

## Product details

In [ ]:
def get_product_data(product_url):
    print(product_url)
    try:
        source = requests.get(product_url)
        sleeptime = random.uniform(2, 5)
        sleep(sleeptime)
        soup = BeautifulSoup(source.content, "html.parser")
        script = soup.find_all('script')
        
        
        try:
        
            p = str(script[1])
            p = p.replace("\\", "")
            start = p.find('''"{''')
            end = p.find('''";''')
            product = p[start+1:end]
            data_item = json.loads(product)
            
            #rating&reviews
            r = str(script[-11])
            r_start = r.find('''{"chatResponsiveRate"''')
            r_end = r.find("promotionTags")
            rating_data_r = r[r_start:r_end-2]
            rating_r = json.loads(rating_data_r)
            
            # get_stars_reviews
            s_start = r.find('''"ratings":''')
            s_end = r.find(''',"reportReasons"''')
            rating_data_s =r[s_start+10:s_end ] 
            rating_s = json.loads(rating_data_s)
            
            # get_diliverytime_fee
            d_start = r.find('''"duringTime":''')
            d_end = r.find(''',"title":"Standard Delivery"''')
            dilivery_d = r[d_start:d_end-1].strip('''"''').split(",")

            
            #category
            try:
                m_cat = data_item['pdt_category'][0]
                main_category_names.append(m_cat)
            except Exception as e:
                print(f"Got exception in main_category: {str(e)}")
                main_category_names.append(" ")
            try:
                s_cat = data_item['pdt_category'][1]
                sub_category_names.append(s_cat)
            except Exception as e:
                print(f"Got exception in sub_category: {str(e)}")
                sub_category_names.append(" ")
            try:
                f_cat = data_item['pdt_category'][2]
                final_category_names.append(f_cat)
            except Exception as e:
                print(f"Got exception in final_category: {str(e)}")
                final_category_names.append(" ")

            #discount

            try:
                discount = data_item['pdt_discount']
                discounts.append(discount)
            except Exception as e:
                print(f"Got exception in discount: {str(e)}")
                discounts.append(" ")

            #brand_name

            try:
                brand_name = data_item['brand_name']
                brand_names.append(brand_name)
            except Exception as e:
                print(f"Got exception in brand_name: {str(e)}")
                brand_names.append(" ")

            #product_sku

            try:
                product_sku = data_item['pdt_sku']
                product_skus.append(product_sku)
            except Exception as e:
                print(f"Got exception in product_sku: {str(e)}")
                product_skus.append(" ")

            #product_names

            try:
                product_name = data_item['pdt_name']
                product_names.append(product_name)
            except Exception as e:
                print(f"Got exception in product_name: {str(e)}")
                product_names.append(" ")


            #product_price

            try:
                product_price = data_item['pdt_price']
                product_prices.append(product_price)
            except Exception as e:
                print(f"Got exception in product_price: {str(e)}")
                product_prices.append(" ")



            #currency_code

            try:
                currency_code = data_item['core']['currencyCode']
                currency_codes.append(currency_code)
            except Exception as e:
                print(f"Got exception in currency_code: {str(e)}")
                currency_codes.append(" ")

            #seller_name

            try:
                seller_name = data_item['seller_name']
                seller_names.append(seller_name)
            except Exception as e:
                print(f"Got exception in seller_name: {str(e)}")
                seller_names.append(" ")


         
            #rating&reviews
            try:
                chat_response = rating_r['chatResponsiveRate']['value']
                chat_responses.append(chat_response)
            except Exception as e:
                print(f"Got exception in chat_response: {str(e)}")
                chat_responses.append(" ")

            try:
                positive_seller = rating_r['positiveSellerRating']['value']
                positive_sellers.append(positive_seller)
            except Exception as e:
                print(f"Got exception in positive_seller: {str(e)}")
                positive_sellers.append(" ")

            try:
                ship_on_time = rating_r['shipOnTime']['value']
                ship_on_times.append(ship_on_time)
            except Exception as e:
                print(f"Got exception in ship_on_time: {str(e)}")
                ship_on_times.append(" ")
               
            # get_stars_reviews
            try:
                star = rating_s['average']
                stars.append(star)
            except Exception as e:
                print(f"Got exception in star: {str(e)}")
                stars.append(" ")

            try:
                review = rating_s['reviewCount']
                reviews.append(review)
            except Exception as e:
                print(f"Got exception in review: {str(e)}")
                reviews.append(" ")

            # get_diliverytime_fee
            try:
                delivery_time = dilivery_d[0][13:-1].strip('''(s)''')
                delivery_times.append(delivery_time)
            except Exception as e:
                print(f"Got exception in delivery time: {str(e)}")
                delivery_times.append(" ")

            try:    
                delivery_fee = dilivery_d[2][11:]
                delivery_fees.append(delivery_fee)
            except Exception as e:
                print(f"Got exception in delivery fee: {str(e)}")
                delivery_fees.append(" ")

        except Exception as e:
            print(f"Got exception in product_data: {str(e)}")
            product_urls.append(product_url)
    except Exception as e:
        print(f"Got exception in product_url: {str(e)}")
        product_urls.append(product_url)
       

In [ ]:
get_sub_category_url()

In [ ]:
get_final_category_url()

In [ ]:
get_product_url()

In [ ]:
for product_url in product_urls:
    get_product_data(product_url)
    
    
